# sklearn中随机梯度下降
参见 自己封装[代码](playML/LinearRegression.py)

科学方法，把所有的样本都看1遍，把所有的信息都考虑进来了



In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
m = 10000
x = np.random.normal(size=m)
X = x.reshape(-1, 1)
y = 4.*x + 3. + np.random.normal(0, 3, size=m)

In [3]:
from playML.LinearRegression import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit_sgd(X, y, n_iters=2)

LinearRegression()

In [4]:
lin_reg.coef_

array([4.03882587])

In [5]:
lin_reg.intercept_

3.0337469167769564

## 真实使用我们自己的SGD

In [7]:
from sklearn import datasets

boston = datasets.load_boston()
X = boston.data
y = boston.target

X = X[y<50.0]
y = y[y<50.0]

In [8]:
from playML.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, seed=666)

In [12]:
from sklearn.preprocessing import StandardScaler
standardScaler = StandardScaler()

standardScaler.fit(X_train)
X_train_standard = standardScaler.transform(X_train)
X_test_standard = standardScaler.transform(X_test)

In [24]:
from playML.LinearRegression import LinearRegression

lin_reg = LinearRegression()
%time lin_reg.fit_sgd(X_train_standard, y_train, n_iters=2)
lin_reg.score(X_test_standard, y_test)

CPU times: user 11.2 ms, sys: 3.73 ms, total: 15 ms
Wall time: 12 ms


0.6365748162891902

In [27]:
from playML.LinearRegression import LinearRegression

lin_reg = LinearRegression()
%time lin_reg.fit_sgd(X_train_standard, y_train, n_iters=50)
lin_reg.score(X_test_standard, y_test)

CPU times: user 180 ms, sys: 6.59 ms, total: 186 ms
Wall time: 197 ms


0.8125581817242616

In [26]:
from playML.LinearRegression import LinearRegression

lin_reg = LinearRegression()
%time lin_reg.fit_sgd(X_train_standard, y_train, n_iters=500)
lin_reg.score(X_test_standard, y_test)

CPU times: user 1.3 s, sys: 22.1 ms, total: 1.32 s
Wall time: 1.36 s


0.8096223299456528

## scikit-learn中的SGD

In [28]:
from sklearn.linear_model import SGDRegressor

In [29]:
sgd_reg = SGDRegressor()
%time sgd_reg.fit(X_train_standard, y_train)
sgd_reg.score(X_test_standard, y_test)

CPU times: user 3.04 ms, sys: 4.13 ms, total: 7.16 ms
Wall time: 4.69 ms


/Users/mgbook/.pyenv/versions/3.6.4rc1/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


0.805097826144864

In [31]:
sgd_reg = SGDRegressor(n_iter=100)
%time sgd_reg.fit(X_train_standard, y_train)
sgd_reg.score(X_test_standard, y_test)

CPU times: user 6.67 ms, sys: 1.55 ms, total: 8.22 ms
Wall time: 6.3 ms


/Users/mgbook/.pyenv/versions/3.6.4rc1/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:130: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.8127074321896776